# Ler PDF (Portable Document Format)
Estudo para verificar qual a melhor biblioteca para leitura do DOM-SP (Diário Oficial do Município de São Paulo).

## Dados de exemplo

In [84]:
# Importação das bibliotecas necessárias
from datetime import date, datetime, timedelta
from os import getcwd, path, listdir, chdir
import datetime
import os

In [87]:
dt_now = date.today()
year_edition_dom = dt_now.year - 1955
head_dom ='São Paulo, ' + str(year_edition_dom) + ' ('
if dt_now.weekday() == 0:
        dt_now = dt_now + timedelta(-2)
        dt_now = dt_now.strftime("%Y-%m-%d")
else:
        dt_now = dt_now.strftime("%Y-%m-%d")


BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR,'data\\files_pdf')
os.chdir(BASE_DIR)
print(f"DATA_DIR: {DATA_DIR}")
list_files_names = os.listdir(DATA_DIR)
file_test = DATA_DIR +'\\'+ list_files_names[2]
SAVE_DIR = "data\\outputs_ler_pdf\\"
print(f"FILE_NAME: {file_test}")

DATA_DIR: c:\Projetos\sindsep-remocao_pmsp\ler_pdf\data\files_pdf
FILE_NAME: c:\Projetos\sindsep-remocao_pmsp\ler_pdf\data\files_pdf\pg_035-5a16426d8c12e74a30aa609cc7563a92704fd873.pdf


## Biblioteca PyMuPDF
[Documentação]('https://pymupdf.readthedocs.io/en/latest/installation.html')

In [88]:
### Instalação
!pip install --upgrade pymupdf

In [89]:
### Importando a biblioteca
import fitz as fz

In [90]:
### Lendo o arquivo
with fz.open(file_test) as file:
        contexts = ''
        for page in file:
                contexts += page.get_text()

In [91]:
### Salvando o arquivo final
with open(f"{SAVE_DIR}output_fitz.txt","w",encoding='utf-8-sig') as output_file:
    output_file.write(contexts)

## Biblioteca PyPDF2
[Documentação]('https://pypi.org/project/PyPDF2/')

In [92]:
# Instalação
!pip install PyPDF2

In [93]:
### Importando as bibliotecas
import re
import PyPDF2
from PyPDF2 import PdfFileReader


In [94]:
### Lendo o arquivo

file = open(file_test, 'rb')
read_pdf = PyPDF2.PdfFileReader(file)       # Faz a leitura usando a biblioteca
number_of_pages = read_pdf.getNumPages()    # pega o numero de páginas
print(f"Número de páginas: {number_of_pages}")
page = read_pdf.getPage(0)                  # lê a primeira página completa
page_content = page.extractText()           # extrai apenas o texto
contexts = ''.join(page_content)              # faz a junção das linhas 

Número de páginas: 1


In [95]:
### Salvando o arquivo final

with open(f"{SAVE_DIR}output_pyPDF2.txt","w",encoding='utf-8-sig') as output_file:
    output_file.write(contexts)

### Merge PDFs

In [99]:
import os
from PyPDF2 import PdfFileReader, PdfFileMerger

pdf_files = [f for f in os.listdir(DATA_DIR) if f.endswith("pdf")]
merger = PdfFileMerger()

for filename in pdf_files:
    merger.append(PdfFileReader(os.path.join(DATA_DIR, filename), "rb"))

merger.write(os.path.join(SAVE_DIR, "output_pyPDF2_merged_full.pdf"))

## Biblioteca Tabula
[Documentação]('https://pypi.org/project/tabula-py/')

In [100]:
### Instalação
!pip install tabula-py

In [101]:
# Importando a biblioteca
import tabula

In [102]:
# Verificando o versão do java python
tabula.environment_info()

Python version:
    3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]
Java version:
    java version "1.8.0_333"
Java(TM) SE Runtime Environment (build 1.8.0_333-b02)
Java HotSpot(TM) 64-Bit Server VM (build 25.333-b02, mixed mode)
tabula-py version: 2.3.0
platform: Windows-10-10.0.22000-SP0
uname:
    uname_result(system='Windows', node='SIND-WKS-PRES', release='10', version='10.0.22000', machine='AMD64')
linux_distribution: ('', '', '')
mac_ver: ('', ('', '', ''), '')
    


In [103]:
#Area do pdf para ser scaneada
area_text = [[42.849,28.172,1209.134,215.329],[42.849,216.314,1210.119,393.621],[42.849,395.591,1210.119,574.868],[42.849,576.838,1207.164,760.055]]

In [104]:
# Lendo o arquivo
dfs = tabula.read_pdf(file_test,pages="all",area=area_text,multiple_tables=True,encoding='utf-8',pandas_options={"header": [0, 1]})
print(f"Quantidade de páginas: {len(dfs)}")

Quantidade de páginas: 4


In [111]:
### Salvando o arquivo final

tabula.convert_into(file_test,SAVE_DIR+"output_tabula.csv" ,pages="all",output_format = "tsv",area=area_text)

In [112]:
### Convertendo todos os arquivos em csv de uma pasta

tabula.convert_into_by_batch(DATA_DIR,output_format="tsv",area=area_text,pages="all")

## Conclusão

- **Biblioteca PyMuPDF:**
    - Ideal para trabalhar com arquivos PDF simples sem paginação.
- **Biblioteca PyPDF2:**
    - Ótima para manipulação de arquivos PDF como mesclar páginas e arquivos.
- **Biblioteca Tabula:**
    - Ótima para conversões em massa de arquivos e conversão de arquivos PDF complexos com paginação e várias colunas.
